In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import numpy as np 
import sklearn
import torch 
import transformers
from tqdm import tqdm
from nlg_eval_via_simi_measures.bary_score import BaryScoreMetric
from nlg_eval_via_simi_measures.depth_score import DepthScoreMetric
from nlg_eval_via_simi_measures.infolm import InfoLM
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
df = pd.read_csv("https://github.com/dig-team/hanna-benchmark-asg/raw/main/hanna_stories_annotations.csv")

In [ ]:
df_unique_human_story=df[df.Model!="Human"][["Human","Story"]].drop_duplicates(keep="first")
df_unique_human_story

In [ ]:
def compute_bary(ref, hypothesis):
    ref,hypothesis=[ref],[hypothesis]
    metric_call = BaryScoreMetric()
    metric_call.prepare_idfs(ref, hypothesis)
    return metric_call.evaluate_batch(ref, hypothesis)["baryscore_W"][0]

In [ ]:
def compute_depthscore(ref,hypothesis):
    metric_call = DepthScoreMetric()
    metric_call.prepare_idfs(ref, hypothesis)
    return metric_call.evaluate_batch(hypothesis,ref)["depth_score"][0]

In [ ]:
def compute_infolmscore(ref,hypothesis):
    ref,hypothesis=[ref],[hypothesis]
    metric = InfoLM()
    #metric.device="cpu"
    #metric.model.to("cpu")
    metric.prepare_idfs(ref, hypothesis)
    return metric.evaluate_batch(hypothesis, ref)["fisher_rao"][0]

In [ ]:
df_mini=df_unique_human_story.head(1)

In [ ]:
df_mini['infolmscore']= df_mini[["Human", "Story"]].apply(lambda x : compute_infolmscore(*x), axis =1)

In [ ]:
# Heavy computation
df_unique_human_story['baryscore']= df_unique_human_story[["Human", "Story"]].apply(lambda x : compute_bary(*x), axis =1)
df_unique_human_story['depthscore']= df_unique_human_story[["Human", "Story"]].apply(lambda x : compute_depthscore(*x), axis =1)
df_unique_human_story['infolmscore']= df_unique_human_story[["Human", "Story"]].apply(lambda x : compute_infolmscore(*x), axis =1)